In [12]:
#pip install pandas


In [13]:
from facebook_scraper import get_posts
import pandas as pd
from tqdm import tqdm

In [14]:

login = ['defow62637@mliok.com']
password = "PasswordLogin"

# create dataframe to store posts
data_frame = pd.DataFrame()

# Get posts from Moms Demand Action -- Clearly this is a gun control group

pro_gun_control_posts = get_posts(
            account = 'MomsDemandAction',
            pages=100,
            page_limit = None,
            credentials=(login[0], password),
            #cookies='cookies.json',
            options={"progress": True, "posts_per_page": 100, "allow_extra_requests": False})

anti_gun_control_posts = get_posts(
            account = 'GunRightsAcrossAmerica',
            pages=100,
            page_limit = None,
            credentials=(login[0], password),
            #cookies='cookies.json',
            options={"progress": True, "posts_per_page": 100, "allow_extra_requests": False})

posts = []
dict = {}

def get_facebook_posts(sentiment):
    if sentiment == 'pro':
        for index, post in enumerate(pro_gun_control_posts):
            print(f'Post Number {index}')
            posts.append(post)
            print(post['text'][:250])
    else:
        for index, post in enumerate(anti_gun_control_posts):
            print(f'Post Number {index}')
            posts.append(post)
            print(post['text'][:250])






In [15]:
def save_posts_list(df, sentiment):
    df.to_csv(f'data/{sentiment}-posts.csv')

# If the data has already been scraped and collected -- Load the Data

In [16]:
def load_posts_list():
    with open('list_data.txt', 'r') as file:
        string = file.read()
        list = eval(string)
        print(len(list))

        print("Successfully Loaded Saved Posts")
        return list

In [17]:
sent = 'against'

In [18]:
get_facebook_posts(sent) # get pro gun control posts


Post Number 0
Alphabet soup can not make laws by changing definitions. #Constitution #2ndamendment #tyranny
Post Number 1
YOUTUBE.COM
Hotel clerk armed with handgun stops rifle-carrying robber, HPD says
Post Number 2
We are at the a pivotal point in our nation's history. Which direction we go in is up to us. If we lose our autonomy gun rights will go with it.

DEFEATTHEMANDATESDC.COM
Defeat The Mandates DC: An American Homecoming
Post Number 3
Young military officer says that Americans can become the enemy and the guns could be pointed at us!

YOUTUBE.COM
Nasty Feminazi threatens U.S. patriots with martial law! #SHORTS
Post Number 4
#2A #constitutionalcarry #concealedcarry #vetooveride contact your state representatives state senators!
Ask them to override the JBE Vetos. This includes constitutional carry, medical freedom bills, ban men from women's sports & more

YOUTUBE.COM
UP
Post Number 5

Post Number 6
Please join us in supporting constitutional carry in Louisiana #2A

NAGR.ORG
Su

In [19]:
data_frame = pd.DataFrame(posts)
data_frame = data_frame[['post_id', 'time', 'text', 'reaction_count']]
if sent == 'pro':
    data_frame['label'] = 1
else:
    data_frame['label'] = 0


In [20]:
import re
def text_preprocessing(text):
    # Remove entity mentions starting with '@user'
    text = re.sub(r'@\w+', '', text)
    
    # Remove URLs
    text = re.sub(r'(http|https)://[^\s]+', '', text)
    
    # Remove text between two asterisks -- Usually emojis/symbols in the dataset
    text = re.sub(r'\*.*?\*', '', text)
    
    # Correct common errors
    text = text.replace('&amp;', '&')
    
    # Remove excess whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Output to CSV file after preprocessing 

In [21]:
# Preprocess text
data_frame['text'] = data_frame['text'].apply(text_preprocessing)

# Remove empty posts
data_frame = data_frame[data_frame['text'] != '']

# Remove duplicate posts
data_frame = data_frame.drop_duplicates(subset='text')

# Remove posts with less than 10 characters
data_frame = data_frame[data_frame['text'].str.len() >= 10]

# Save posts
save_posts_list(data_frame, sent)